# Modelo de Datos - Tercer Entrega

###### Angeles Aurora Guzmán Luna
###### Matrícula: 201937787

Primero importamos las librerías necesarias para el análisis.

In [1]:
import pandas as pd
import json
import requests

# Tablas

### Races

Cramos la tabla "Races"

In [2]:
#Creamos un diccionario para despues almacenar lo extraido de la API.
races = {'year':[],
         'round':[],
         'raceName':[],
         'circuitId':[],
         'lat':[],
         'long':[],
         'country':[],
         'date':[],
         'url':[]}

#Hacemos ciclo para cambiar el año de cada URL.
for year in list(range(2010,2023)): 
    url = 'http://ergast.com/api/f1/{}.json?limit=1000' #Llamamos el URL de la página.
    response = requests.get(url.format(year)) #Para verificar la disponibilidad de la página.
    content = json.loads(response.content) #Para ver el contenido de la página.
    for item in content['MRData']['RaceTable']['Races']: #Usamos otro ciclo for para para extraer cada elemento de la lista de nuestra API y guardarlo en el diccionario.
        races['year'].append(int(item['season']))
        races['round'].append(int(item['round']))
        races['raceName'].append(item['raceName'])
        races['circuitId'].append(item['Circuit']['circuitId'])
        races['lat'].append(float(item['Circuit']['Location']['lat']))
        races['long'].append(float(item['Circuit']['Location']['long']))
        races['country'].append(item['Circuit']['Location']['country'])
        races['date'].append(item['date'])
        races['url'].append(item['url'])

#Convertimos nuestro diccionario en un DataFrame
df_races = pd.DataFrame(races)

#Cambiamos el tipo de datos de la columna 'Date' para convertirlo en fecha.
df_races['date'] = df_races['date'].astype('datetime64[ns]') 
print(df_races)

#Exportamos el DataFrame en formato csv
df_races.to_csv('races.csv',index = False)

     year  round                  raceName    circuitId       lat       long  \
0    2010      1        Bahrain Grand Prix      bahrain  26.03250   50.51060   
1    2010      2     Australian Grand Prix  albert_park -37.84970  144.96800   
2    2010      3      Malaysian Grand Prix       sepang   2.76083  101.73800   
3    2010      4        Chinese Grand Prix     shanghai  31.33890  121.22000   
4    2010      5        Spanish Grand Prix    catalunya  41.57000    2.26111   
..    ...    ...                       ...          ...       ...        ...   
254  2022     18       Japanese Grand Prix       suzuka  34.84310  136.54100   
255  2022     19  United States Grand Prix     americas  30.13280  -97.64110   
256  2022     20    Mexico City Grand Prix    rodriguez  19.40420  -99.09070   
257  2022     21      Brazilian Grand Prix   interlagos -23.70360  -46.69970   
258  2022     22      Abu Dhabi Grand Prix   yas_marina  24.46720   54.60310   

       country       date              

### Seasons

Cramos la tabla "Seasons"

In [3]:
#Usamos ?limit=500 para tener todos los datos y no solo los primeros 30. 
url='http://ergast.com/api/f1/seasons.json?limit=500' 

#Para verificar la disponibilidad de la página.
response = requests.get(url)
response 

#Para ver el contenido de la página.
content = json.loads(response.content) 

#Las variables/columnas que se utilizaremos.
season = {'year':[],
          'url':[]} 

#Usamos un ciclo for para extraer cada elemento de la lista de nuestra API y lo guardamos en nuestro diccionario.
for i in content['MRData']['SeasonTable']['Seasons']: 
    season['year'].append(int(i['season']))
    season['url'].append(i['url'])

#Convertimos nuestro diccionario en un DataFrame
df_Season=pd.DataFrame(season) 
print(df_Season)

#Exportamos el DataFrame en formato csv
df_Season.to_csv('seasons.csv',index=False)

    year                                                url
0   1950  http://en.wikipedia.org/wiki/1950_Formula_One_...
1   1951  http://en.wikipedia.org/wiki/1951_Formula_One_...
2   1952  http://en.wikipedia.org/wiki/1952_Formula_One_...
3   1953  http://en.wikipedia.org/wiki/1953_Formula_One_...
4   1954  http://en.wikipedia.org/wiki/1954_Formula_One_...
..   ...                                                ...
69  2019  http://en.wikipedia.org/wiki/2019_Formula_One_...
70  2020  http://en.wikipedia.org/wiki/2020_Formula_One_...
71  2021  http://en.wikipedia.org/wiki/2021_Formula_One_...
72  2022  http://en.wikipedia.org/wiki/2022_Formula_One_...
73  2023  https://en.wikipedia.org/wiki/2023_Formula_One...

[74 rows x 2 columns]


### Drivers

Cramos la tabla "Drivers"

In [4]:
#Creamos un diccionario para despues almacenar lo extraido de la API.
drivers = {'year':[],
           "DriverId":[],
           "Number":[],
           "code":[],
           "forename":[],
           "surname":[],
           "dob":[],
           "nationality":[],
           "url":[]}

#Como en la tabla 'Races', creamos dos ciclos, uno que nos cambie el año de cada URL y otro que extraiga la información 
#para guardarla en el diccionario, sin embargo, como en nuestros datos existen NA, hacemos una modificación.
for year in list(range(2010,2023)):
    url = 'http://ergast.com/api/f1/{}/drivers.json?limit=1000'
    response = requests.get(url.format(year))
    content = json.loads(response.content)
    for i in range(0,len(content['MRData']['DriverTable']["Drivers"])):
            drivers["year"].append(int(content['MRData']['DriverTable']['season']))
            try: #Usamos try para tratar de encontar esa llave o información.
                drivers["Number"].append(int(content['MRData']['DriverTable']['Drivers'][i]["permanentNumber"]))
            except: #En caso de que try no encuentra la información, usamos excep, que deja vacio ese espacio.
                drivers["Number"].append("")
            drivers["DriverId"].append(content['MRData']['DriverTable']['Drivers'][i]["driverId"])
            try:
                drivers["code"].append(content['MRData']['DriverTable']['Drivers'][i]["code"])
            except:
                drivers["code"].append("")
            drivers["forename"].append(content['MRData']['DriverTable']['Drivers'][i]["givenName"])
            drivers["surname"].append(content['MRData']['DriverTable']['Drivers'][i]["familyName"])                       
            drivers["dob"].append(content['MRData']['DriverTable']['Drivers'][i]["dateOfBirth"])                        
            drivers["nationality"].append(content['MRData']['DriverTable']['Drivers'][i]["nationality"])
            drivers["url"].append(content['MRData']['DriverTable']['Drivers'][i]["url"]) 

#Convertimos nuestro diccionario en un DataFrame
df_drivers = pd.DataFrame(drivers)

#Cambiamos el tipo de datos de la columna 'Dob' para convertirlo en fecha.
df_drivers['dob'] = df_drivers['dob'].astype('datetime64[ns]') 
print(df_drivers)

#Exportamos el DataFrame en formato csv
df_drivers.to_csv('drivers.csv',index=False)


     year        DriverId Number code   forename      surname        dob  \
0    2010     alguersuari         ALG      Jaime  Alguersuari 1990-03-23   
1    2010          alonso     14  ALO   Fernando       Alonso 1981-07-29   
2    2010     barrichello         BAR     Rubens  Barrichello 1972-05-23   
3    2010           buemi         BUE  Sébastien        Buemi 1988-10-31   
4    2010          button     22  BUT     Jenson       Button 1980-01-19   
..    ...             ...    ...  ...        ...          ...        ...   
299  2022          stroll     18  STR      Lance       Stroll 1998-10-29   
300  2022         tsunoda     22  TSU       Yuki      Tsunoda 2000-05-11   
301  2022  max_verstappen     33  VER        Max   Verstappen 1997-09-30   
302  2022          vettel      5  VET  Sebastian       Vettel 1987-07-03   
303  2022            zhou     24  ZHO     Guanyu         Zhou 1999-05-30   

    nationality                                                url  
0       Spanish   

### Constructors

Cramos la tabla "Constructors"

In [5]:
#Creamos un diccionario para despues almacenar lo extraido de la API.
constructors={"year":[],
              "constuctorId":[],
              "name":[],
              "nationality":[],
              "url":[],}

#Como en la tabla 'Races', creamos dos ciclos, uno que nos cambie el año de cada URL y otro que extraiga la información 
#para guardarla en el diccionario.
for year in list(range(2010,2023)):
    url='http://ergast.com/api/f1/{}/constructors.json?limit=1000'
    response = requests.get(url.format(year))
    content=json.loads(response.content)
    for item in range(0,len(content['MRData']['ConstructorTable']['Constructors'])):
        constructors["year"].append(int(content['MRData']['ConstructorTable']['season']))
        constructors["constuctorId"].append(content['MRData']['ConstructorTable']['Constructors'][item]['constructorId'])
        constructors['name'].append(content['MRData']['ConstructorTable']['Constructors'][item]['name'])
        constructors['nationality'].append(content['MRData']['ConstructorTable']['Constructors'][item]['nationality'])
        constructors['url'].append(content['MRData']['ConstructorTable']['Constructors'][item]['url'])


#Convertimos nuestro diccionario en un DataFrame
df_constructors=pd.DataFrame(constructors) 
print(df_constructors)

#Exportamos el DataFrame en formato csv
df_constructors.to_csv('constructors.csv',index=False)

     year  constuctorId          name nationality  \
0    2010       ferrari       Ferrari     Italian   
1    2010   force_india   Force India      Indian   
2    2010           hrt           HRT     Spanish   
3    2010  lotus_racing         Lotus   Malaysian   
4    2010       mclaren       McLaren     British   
..    ...           ...           ...         ...   
134  2022          haas  Haas F1 Team    American   
135  2022       mclaren       McLaren     British   
136  2022      mercedes      Mercedes      German   
137  2022      red_bull      Red Bull    Austrian   
138  2022      williams      Williams     British   

                                                   url  
0        http://en.wikipedia.org/wiki/Scuderia_Ferrari  
1    http://en.wikipedia.org/wiki/Racing_Point_Forc...  
2         http://en.wikipedia.org/wiki/Hispania_Racing  
3            http://en.wikipedia.org/wiki/Lotus_Racing  
4                 http://en.wikipedia.org/wiki/McLaren  
..                   

### Circuits

Creamos la tabla "Circuits"

In [6]:
#Creamos un diccionario para despues almacenar lo extraido de la API.
circuits={'year':[],
          'circuitId':[],
          'name':[],
          'location':[], 
          'country':[],
          'lat':[], 
          'lng':[], 
          'alt':[],
          'url':[]}

#Como en la tabla 'Races', creamos dos ciclos, uno que nos cambie el año de cada URL y otro que extraiga la información 
#para guardarla en el diccionario.
for year in list(range(2010,2023)):
    url= 'http://ergast.com/api/f1/{}/circuits.json?limit=1000'
    response = requests.get(url.format(year)) 
    content=json.loads(response.content)
    path=content['MRData']['CircuitTable']
    for item in range(0,len(path['Circuits'])):
        circuits['year'].append(int(path['season']))
        circuits['circuitId'].append(path['Circuits'][item]['circuitId'])
        circuits['name'].append(path['Circuits'][item]['circuitName'])
        circuits['location'].append(path['Circuits'][item]['Location']['locality'])
        circuits['country'].append(path['Circuits'][item]['Location']['country'])
        circuits['lat'].append(float(path['Circuits'][item]['Location']['lat']))
        circuits['lng'].append(float(path['Circuits'][item]['Location']['long']))
        circuits['alt'].append(path['Circuits'][item]['Location']['locality'])
        circuits['url'].append(path['Circuits'][item]['url'])

#Convertimos nuestro diccionario en un DataFrame
df_circuits=pd.DataFrame(circuits)
print(df_circuits)

#Exportamos el DataFrame en formato csv
df_circuits.to_csv('circuits.csv',index=False)

     year       circuitId                            name    location  \
0    2010     albert_park  Albert Park Grand Prix Circuit   Melbourne   
1    2010         bahrain   Bahrain International Circuit      Sakhir   
2    2010       catalunya  Circuit de Barcelona-Catalunya    Montmeló   
3    2010  hockenheimring                  Hockenheimring  Hockenheim   
4    2010     hungaroring                     Hungaroring    Budapest   
..    ...             ...                             ...         ...   
250  2022             spa    Circuit de Spa-Francorchamps         Spa   
251  2022          suzuka                  Suzuka Circuit      Suzuka   
252  2022      villeneuve       Circuit Gilles Villeneuve    Montreal   
253  2022      yas_marina              Yas Marina Circuit   Abu Dhabi   
254  2022       zandvoort          Circuit Park Zandvoort   Zandvoort   

         country      lat        lng         alt  \
0      Australia -37.8497  144.96800   Melbourne   
1        Bahrain  2

### Qualys

Creamos la tabla "Qualys"

In [7]:
#Creamos un diccionario para despues almacenar lo extraido de la API.
quali={"year":[],
       "driverId":[],
       "ConstructorId":[],
       "number":[],
       "position":[],
       "qualifyId":[],
       "raceId":[],
       "q1":[],
       "q2":[],
       "q3":[]}

#Creamos ciclo para cambiar el año de cada URL.
for year in range(2010,2023):
    #Dividimos el URL en año y número de carrera.
    url='http://ergast.com/api/f1' 
    url1=url+'/{}.json?limit=1000'
    url2=url+'/{}'.format(year) #Concatenamos la URL para cambiar el año de cada link.
    response = requests.get(url1.format(year)) #Para verificar la disponibilidad de la página.
    content=json.loads(response.content) #Para ver el contenido de la página.
    link=[] #Creamos una lista para guardar los lnks de todos los años con sus respectivas carreras.
    for k in range(1,len(content['MRData']['RaceTable']['Races'])+1):
        url3=url2+'/{}/qualifying.json?limit=100' #Concatenamos los links para ir cambiando las carreras de cada año.
        link.append(url3.format(k)) #Agregamos este URL a la lista de links.
    for lk in link: #Usamos un ciclo for para extraer la información de cada link, es decir, se extrae la información de cada año y de cada carrera.
        response = requests.get(lk)
        content=json.loads(response.content)
        for it in content['MRData']['RaceTable']['Races']: #Aqui usamos dos ciclos for, uno para el año y otro para la carrera.
            for it1 in content['MRData']['RaceTable']['Races'][0]['QualifyingResults']:
                quali["year"].append(int(it['season']))
                quali["qualifyId"].append(int( it['round']))
                quali["raceId"].append(it['raceName']) 
                quali["driverId"].append(it1['Driver']['driverId'])   
                quali["ConstructorId"].append(it1['Constructor']['constructorId'])
                quali["number"].append(int(it1['number']))
                quali["position"].append(int(it1['position']))
                quali["q1"].append(it1['Q1']) #Eliminamos los ultimos 5.
                #Usamos el mismo proceso que en la tabla 'Driver' de try y except para los datos NA.
                try: 
                    quali["q2"].append(it1['Q2']) #Volvemos a eliminar a los ultimos 5
                except:
                    quali["q2"].append("")
                try:
                    quali["q3"].append(it1['Q3']) #Tenemos nuestro top10
                except:
                    quali["q3"].append("")

#convertimos a DataFrame nuestro diccionario.
dfquali=pd.DataFrame.from_dict(quali,orient='index')
dfquali=dfquali.transpose() #Lo transponemos
print(dfquali.head(30))

#Exportamos el DataFrame en formato csv
dfquali.to_csv('qualifiying.csv',index=False)

    year            driverId ConstructorId number position qualifyId  \
0   2010              vettel      red_bull      5        1         1   
1   2010               massa       ferrari      7        2         1   
2   2010              alonso       ferrari      8        3         1   
3   2010            hamilton       mclaren      2        4         1   
4   2010             rosberg      mercedes      4        5         1   
5   2010              webber      red_bull      6        6         1   
6   2010  michael_schumacher      mercedes      3        7         1   
7   2010              button       mclaren      1        8         1   
8   2010              kubica       renault     11        9         1   
9   2010               sutil   force_india     14       10         1   
10  2010         barrichello      williams      9       11         1   
11  2010              liuzzi   force_india     15       12         1   
12  2010          hulkenberg      williams     10       13      

### Sprint results

In [8]:
#Creamos un diccionario para despues almacenar lo extraido de la API.
sprintResult={"year":[],"raceId":[],
              "driverId":[],"round":[],
              "constructorId":[],"number":[],
              "grid":[],"position":[],
              "positionText":[],"points":[],
              "laps":[],"time":[],
              "miliseconds":[],"fastestLap":[],
              "statusId":[],"fastestLapTime":[]}

#Creamos ciclo para cambiar el año de cada URL.
for year in range(2010,2023):
    #Dividimos el URL en año y número de carrera.
    url='http://ergast.com/api/f1' 
    url1=url+'/{}.json?limit=1000'
    url3=url+'/{}'.format(year)
    response = requests.get(url1.format(year)) #Para verificar la disponibilidad de la página.
    content=json.loads(response.content) #Para ver el contenido de la página.
    urls=[] #Creamos una lista para guardar los lnks de todos los años con sus respectivas carreras.
    for i in range(1,len(content['MRData']['RaceTable']['Races'])+1): #Aqui usamos dos ciclos for, uno para el año y otro para la carrera.
        url2=url3+'/{}/sprint.json?limit=1000'
        response = requests.get(url2.format(i))
        content=json.loads(response.content) 
        urls.append(url2.format(i)) #Vamos agregando estos URL en los de cada año.
    i=0 
    for j in urls: #Ahora de cada URL extraemos la información que necesitamos
        response = requests.get(j)
        content=json.loads(response.content)
        
        try: #Usamos el mismo proceso que en la tabla 'Driver' de try y except para los datos NA.
            for item in content['MRData']['RaceTable']['Races'][0]['SprintResults']:
                sprintResult["year"].append(int(content['MRData']['RaceTable']['Races'][0]['season']))
                sprintResult["driverId"].append(item["Driver"]["driverId"])
                sprintResult["raceId"].append(content['MRData']['RaceTable']['Races'][0]['raceName'])
                sprintResult["round"].append(int(content['MRData']['RaceTable']['Races'][0]['round']))
                sprintResult["constructorId"].append(item["Constructor"]["constructorId"])
                sprintResult["number"].append(int(item['Driver']['permanentNumber']))
                sprintResult["grid"].append(int(item["grid"]))
                sprintResult["position"].append(int(item['position']))
                sprintResult["positionText"].append(item['positionText'])
                sprintResult["points"].append(int(item['points']))
                sprintResult["laps"].append(int(item['laps']))
                try:
                    sprintResult["time"].append(item["Time"]["time"])
                except:
                    sprintResult["time"].append(0)
                try:
                    sprintResult["miliseconds"].append(item["Time"]["millis"])
                except:
                    sprintResult["miliseconds"].append("")
                try:    
                    sprintResult["fastestLap"].append(item['FastestLap']["lap"])
                except:
                    sprintResult["fastestLap"].append("")
                try:    
                    sprintResult["fastestLapTime"].append(item['FastestLap']['Time']['time'])
                except:
                    sprintResult["fastestLapTime"].append("")
                try:    
                    sprintResult["statusId"].append(item['status'])
                except:
                    sprintResult["statusId"].append("")
        except:
            asd=1
                
#Convertimos a DataFrame nuestro diccionario.               
SprintResult=pd.DataFrame.from_dict(sprintResult,orient='index')
SprintResult=SprintResult.transpose() #Lo transponemos

print(SprintResult.head())

#Exportamos el DataFrame en formato csv
SprintResult.to_csv('SprintResult.csv')

   year              raceId        driverId round constructorId number grid  \
0  2021  British Grand Prix  max_verstappen    10      red_bull     33    2   
1  2021  British Grand Prix        hamilton    10      mercedes     44    1   
2  2021  British Grand Prix          bottas    10      mercedes     77    3   
3  2021  British Grand Prix         leclerc    10       ferrari     16    4   
4  2021  British Grand Prix          norris    10       mclaren      4    6   

  position positionText points laps       time miliseconds fastestLap  \
0        1            1      3   17  25:38.426     1538426         14   
1        2            2      2   17     +1.430     1539856         17   
2        3            3      1   17     +7.502     1545928         17   
3        4            4      0   17    +11.278     1549704         16   
4        5            5      0   17    +24.111     1562537         16   

   statusId fastestLapTime  
0  Finished       1:30.013  
1  Finished       1:29.937  

### Status

In [9]:
#Creamos un diccionario para despues almacenar lo extraido de la API.
status={"year":[],
        "statusId":[],
        "status":[]}

#Creamos ciclo para cambiar el año de cada URL.
for year in range(2010,2023):
    url='http://ergast.com/api/f1/{}/status.json'
    response = requests.get(url.format(year))
    content=json.loads(response.content)
    for item in content['MRData']['StatusTable']["Status"]: #Aqui usamos dos ciclos for, uno para el año y otro para la carrera.
        status["statusId"].append(item['statusId'])
        status["status"].append(item['status'])
        status["year"].append(content['MRData']['StatusTable']['season'])

#Convertimos a DataFrame nuestro diccionario.
statuss=pd.DataFrame(status)

print(statuss.head())

#Exportamos el DataFrame en formato csv
statuss.to_csv('status.csv')

   year statusId     status
0  2010        1   Finished
1  2010        3   Accident
2  2010        4  Collision
3  2010        5     Engine
4  2010        6    Gearbox


### Results

In [10]:
#Creamos un diccionario para despues almacenar lo extraido de la API.
resultados={"year":[],"raceId":[],
          "driverId":[],"constructorId":[],
          "number":[],"grid":[],
          "position":[],"positionText":[],
          "points":[],"laps":[],
          "time":[],"miliseconds":[],
          "fastestLap":[],"fastestLapSpeed":[],
          "statusId":[],"fastestLapTime":[],
          "round":[]}

#Creamos ciclo para cambiar el año de cada URL.
for year in range(2010,2023):
    url='http://ergast.com/api/f1' 
    url1=url+'/{}.json?limit=1000'
    url3=url+'/{}'.format(year)
    response = requests.get(url1.format(year))
    content=json.loads(response.content)
    tampath=len(content['MRData']['RaceTable']['Races'])
    links=[]
    for i in range(1,tampath+1): #Aqui usamos dos ciclos for, uno para el año y otro para la carrera.
        url2=url3+'/{}/results.json?limit=1000'
        response = requests.get(url2.format(i))
        content=json.loads(response.content)
        links.append(url2.format(i))
    i=0 
    for j in links:
        response = requests.get(j)
        content=json.loads(response.content)
        
        for item in content['MRData']['RaceTable']['Races'][0]['Results']:
            resultados["year"].append(int(content['MRData']['RaceTable']['Races'][0]['season']))
            resultados["raceId"].append(content['MRData']['RaceTable']['Races'][0]['raceName'])
            resultados["round"].append(int(content['MRData']['RaceTable']['Races'][0]['round']))
            resultados["driverId"].append(item['Driver']['driverId'])
            resultados["constructorId"].append(item['Constructor']['constructorId'])
            try: #Usamos el mismo proceso que en la tabla 'Driver' de try y except para los datos NA.
                resultados["number"].append(item['Driver']['permanentNumber'])
            except:
                resultados["number"].append("")
            resultados["grid"].append(item['grid'])
            resultados["position"].append(int(item['position']))
            resultados["positionText"].append(item['positionText'])
            resultados["points"].append(float(item['points']))
            resultados["laps"].append(int(item['laps']))
            try:
                resultados["time"].append(item['Time']['time'])
            except:
                resultados["time"].append("")
            try:
                resultados["miliseconds"].append(int(item['Time']['millis']))
            except: 
                resultados["miliseconds"].append("")
            try:
                resultados["fastestLap"].append(int(item['FastestLap']['lap']))
            except: 
                resultados["fastestLap"].append("")
            try:
                resultados["fastestLapTime"].append(item['FastestLap']['Time']['time'])
            except: 
                resultados["fastestLapTime"].append("")
            try:
                resultados["fastestLapSpeed"].append(item['FastestLap']['AverageSpeed']['speed'])
            except: 
                resultados["fastestLapSpeed"].append("")
                
            try:
                resultados["statusId"].append(item['status'])
            except: 
                resultados["statusId"].append("")
                
#Convertimos a DataFrame nuestro diccionario.
Resultados=pd.DataFrame.from_dict(resultados,orient='index')
Resultados=Resultados.transpose() #Transponemos       

print(Resultados.head())

#Exportamos el DataFrame en formato csv
Resultados.to_csv('Results.csv')

   year              raceId  driverId constructorId number grid position  \
0  2010  Bahrain Grand Prix    alonso       ferrari     14    3        1   
1  2010  Bahrain Grand Prix     massa       ferrari     19    2        2   
2  2010  Bahrain Grand Prix  hamilton       mclaren     44    4        3   
3  2010  Bahrain Grand Prix    vettel      red_bull      5    1        4   
4  2010  Bahrain Grand Prix   rosberg      mercedes      6    5        5   

  positionText points laps         time miliseconds fastestLap  \
0            1   25.0   49  1:39:20.396     5960396         45   
1            2   18.0   49      +16.099     5976495         38   
2            3   15.0   49      +23.182     5983578         42   
3            4   12.0   49      +38.799     5999195         32   
4            5   10.0   49      +40.213     6000609         45   

  fastestLapSpeed  statusId fastestLapTime round  
0         191.706  Finished       1:58.287     1  
1         189.392  Finished       1:59.732  

### PitStops

In [11]:
#Creamos un diccionario para despues almacenar lo extraido de la API.
pitStops = {'year':[],
            "raceId":[],
            'driverId':[], 
            'stop':[],
            'lap':[], 
            'time':[],
            'round':[], 
            'duration':[]}

#Creamos ciclo para cambiar el año de cada URL.
for year in range(2010,2023):
    url='http://ergast.com/api/f1' 
    url1=url+'/{}.json?limit=1000'
    url3=url+'/{}'.format(year)
    response = requests.get(url1.format(year))
    content=json.loads(response.content)
    tampath=len(content['MRData']['RaceTable']['Races'])
    links=[]
    for i in range(1,tampath+1):  #Aqui usamos dos ciclos for, uno para el año y otro para la carrera.
        url2=url3+'/{}/pitstops.json?limit=1000'
        response = requests.get(url2.format(i))
        content=json.loads(response.content)
        links.append(url2.format(i))
    i=0 
    for j in links:
        try: #Usamos el mismo proceso que en la tabla 'Driver' de try y except para los datos NA.
            response = requests.get(j)
            content=json.loads(response.content)
        except:
            print(f"Hay un error en {j}")
        try: 
            content['MRData']['RaceTable']['Races'][0]['PitStops']
            for item in content['MRData']['RaceTable']['Races'][0]['PitStops']:
                pitStops["raceId"].append(content['MRData']['RaceTable']['Races'][0]['raceName'])
                pitStops["round"].append(int(content['MRData']['RaceTable']['Races'][0]['round']))
                pitStops["year"].append(int(content['MRData']['RaceTable']['Races'][0]['season']))
                pitStops["driverId"].append(item['driverId'])
                pitStops["stop"].append(int(item['stop']))
                pitStops["lap"].append(int(item['lap']))
                pitStops["time"].append(item['time'])
                pitStops["duration"].append(item['duration'])
            
        except:
            asd=1
            
        
#Convertimos a DataFrame nuestro diccionario.
pitStopss=pd.DataFrame(pitStops)   

print(pitStopss.head())

#Exportamos el DataFrame en formato csv
pitStopss.to_csv('pitStops.csv')

   year                 raceId            driverId  stop  lap      time  \
0  2011  Australian Grand Prix         alguersuari     1    1  17:05:23   
1  2011  Australian Grand Prix  michael_schumacher     1    1  17:05:52   
2  2011  Australian Grand Prix              webber     1   11  17:20:48   
3  2011  Australian Grand Prix              alonso     1   12  17:22:34   
4  2011  Australian Grand Prix               massa     1   13  17:24:10   

   round duration  
0      1   26.898  
1      1   25.021  
2      1   23.426  
3      1   23.251  
4      1   23.842  


### LapTimes

In [12]:
#Creamos un diccionario para despues almacenar lo extraido de la API.
lapTimes={"year":[],
          "round":[],
          "raceId":[],
          "driverId":[],
          "lap":[],
          "position":[],
          "time":[]}

#Creamos ciclo para cambiar el año de cada URL.
for year in range(2010,2023):
    url='http://ergast.com/api/f1' 
    url1=url+'/{}.json?limit=1000'
    url3=url+'/{}'.format(year)
    response = requests.get(url1.format(year))
    content=json.loads(response.content)
    links=[]
    for i in range(1,len(content['MRData']['RaceTable']['Races'])+1):  #Aqui usamos dos ciclos for, uno para el año y este para la carrera.
        url2=url3+'/{}/laps.json?limit=10000'
        response = requests.get(url2.format(i))
        content=json.loads(response.content)
        links.append(url2.format(i))
    for j in links:
        response = requests.get(j)
        content=json.loads(response.content)
        for i in content['MRData']['RaceTable']["Races"][0]["Laps"]:
            for item in i['Timings']:
                try: #Usamos el mismo proceso que en la tabla 'Driver' de try y except para los datos NA.
                    lapTimes["year"].append(content['MRData']['RaceTable']['Races'][0]['season'])                    
                except:
                    lapTimes["year"].append("")
                try:
                    lapTimes["round"].append(content['MRData']['RaceTable']['Races'][0]['round'])                    
                except:
                    lapTimes["round"].append("")
                try:
                    lapTimes["raceId"].append(content['MRData']['RaceTable']["Races"][0]['raceName'])
                except:
                    lapTimes["raceId"].append("")
                try:
                    lapTimes["driverId"].append(item['driverId'])
                except:
                    lapTimes["driverId"].append("")
                try:
                    lapTimes["lap"].append(i['number'])
                except:
                    lapTimes["lap"].append("")
                try:
                    lapTimes["position"].append(item['position'])
                except:
                    lapTimes["position"].append("")
                try:
                    lapTimes["time"].append(item['time'])
                except:
                    lapTimes["time"].append("")

#Convertimos a DataFrame nuestro diccionario.
lapTimes_df=pd.DataFrame.from_dict(lapTimes,orient='index')

#Lo transponemos
lapTimes_df=lapTimes_df.transpose()
print(lapTimes_df.head())

#Convertimos a DataFrame nuestro diccionario.
lapTimes_df.to_csv('lapTimes.csv',index=False)


   year round              raceId  driverId lap position      time
0  2010     1  Bahrain Grand Prix    vettel   1        1  2:03.366
1  2010     1  Bahrain Grand Prix    alonso   1        2  2:05.394
2  2010     1  Bahrain Grand Prix     massa   1        3  2:06.758
3  2010     1  Bahrain Grand Prix   rosberg   1        4  2:08.063
4  2010     1  Bahrain Grand Prix  hamilton   1        5  2:08.558


### DriverStandings

In [13]:
#Creamos un diccionario para despues almacenar lo extraido de la API.
driverStandings={"year":[],
                 "round":[],
                 "driverId":[],
                 "position":[],
                 "points":[],
                 "positionText":[],
                 "wins":[]}

#Creamos ciclo para cambiar el año de cada URL.
for year in range(2010,2023):
    url='http://ergast.com/api/f1' 
    url1=url+'/{}.json?limit=1000'
    url3=url+'/{}'.format(year)
    response = requests.get(url1.format(year))
    content=json.loads(response.content)
    links=[]
    for i in range(1,len(content['MRData']['RaceTable']['Races'])+1):  #Aqui usamos dos ciclos for, uno para el año y este para la carrera.
        url2=url3+'/{}/driverStandings.json?limit=10000'
        response = requests.get(url2.format(i))
        content=json.loads(response.content)
        links.append(url2.format(i))
    for j in links:
        response = requests.get(j)
        content=json.loads(response.content)
        for item in content['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']:
            driverStandings["year"].append(content['MRData']['StandingsTable']['StandingsLists'][0]['season'])
            driverStandings["round"].append(content['MRData']['StandingsTable']['StandingsLists'][0]['round'])
            driverStandings["driverId"].append(item['Driver']['driverId'])
            driverStandings["position"].append(item['position'])
            driverStandings["points"].append(item['points'])
            driverStandings["wins"].append(item['wins'])
            driverStandings["positionText"].append(item['positionText'])

#Convertimos a DataFrame nuestro diccionario.
driverStandings_df=pd.DataFrame.from_dict(driverStandings,orient='index')

#Lo transponemos
driverStandings_df=driverStandings_df.transpose()
print(driverStandings_df.head())

#Convertimos a DataFrame nuestro diccionario.
driverStandings_df.to_csv('driverStandings.csv',index=False)

   year round  driverId position points positionText wins
0  2010     1    alonso        1     25            1    1
1  2010     1     massa        2     18            2    0
2  2010     1  hamilton        3     15            3    0
3  2010     1    vettel        4     12            4    0
4  2010     1   rosberg        5     10            5    0


### ConstructorStandings

In [14]:
#Creamos un diccionario para despues almacenar lo extraido de la API.
constructorStandings={"year":[],
                      "round":[],
                      "constructorId":[],
                      "points":[],
                      "position":[],
                      "positionText":[],
                      "wins":[]}

#Creamos ciclo para cambiar el año de cada URL.
for year in range(2010,2023):
    url='http://ergast.com/api/f1' 
    url1=url+'/{}.json?limit=1000'
    url3=url+'/{}'.format(year)
    response = requests.get(url1.format(year))
    content=json.loads(response.content)
    links=[]
    for i in range(1,len(content['MRData']['RaceTable']['Races'])+1): #Aqui usamos dos ciclos for, uno para el año y este para la carrera.
        url2=url3+'/{}/constructorStandings.json?limit=10000'
        response = requests.get(url2.format(i))
        content=json.loads(response.content)
        links.append(url2.format(i))
    for j in links:
        response = requests.get(j)
        content=json.loads(response.content)
        for item in content['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings']:
            constructorStandings["year"].append(content['MRData']['StandingsTable']['StandingsLists'][0]["season"])
            constructorStandings["round"].append(content['MRData']['StandingsTable']['StandingsLists'][0]["round"])
            constructorStandings["constructorId"].append(item['Constructor']['constructorId'])
            constructorStandings["points"].append(item['points'])
            constructorStandings["position"].append(item["position"])
            constructorStandings["positionText"].append(item['positionText'])
            constructorStandings["wins"].append(item['wins'])

#Convertimos a DataFrame nuestro diccionario.
constructorStandings_df=pd.DataFrame.from_dict(constructorStandings,orient='index')

#Lo transponemos
constructorStandings_df=constructorStandings_df.transpose()
print(constructorStandings_df.head())

#Convertimos a DataFrame nuestro diccionario.
constructorStandings_df.to_csv('constructorStandings.csv',index=False)

   year round constructorId points position positionText wins
0  2010     1       ferrari     43        1            1    1
1  2010     1       mclaren     21        2            2    0
2  2010     1      mercedes     18        3            3    0
3  2010     1      red_bull     16        4            4    0
4  2010     1   force_india      2        5            5    0


### ConstructorResults

In [15]:
# Separamos la tabla 'Results' y agrupamos las variables para despues sumar los puntos
ConstructorResults=Resultados.groupby(["year","round","raceId",'constructorId','statusId']).agg({'points':"sum"})

# Exportamos nuestro nuevo archivo csv
ConstructorResults.to_csv('ConstructorResults.csv')
print(ConstructorResults.head(10))

                                                        points
year round raceId             constructorId statusId          
2010 1     Bahrain Grand Prix ferrari       Finished      43.0
                              force_india   Finished       2.0
                              hrt           Accident       0.0
                                            Overheating    0.0
                              lotus_racing  +2 Laps        0.0
                                            Hydraulics     0.0
                              mclaren       Finished      21.0
                              mercedes      Finished      18.0
                              red_bull      Finished      16.0
                              renault       Finished       0.0
